In [2]:
import torch
from torch import nn
import pandas as pd
import sklearn
import numpy as np
from sklearn.utils import shuffle
import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sn

In [3]:
def visualize_result(y_true,y_pred):
  cm = confusion_matrix(y_true, y_pred)
  df_cm = pd.DataFrame(cm, range(2), range(2))
  plt.figure(figsize=(10,7))
  sn.set(font_scale=1) 
  sn.heatmap(df_cm, annot=True, annot_kws={"size": 20},fmt="d",cmap="YlGnBu") # font size
  from sklearn.metrics import classification_report
  from sklearn.metrics import f1_score,recall_score,precision_score
  target_names = ['class 0', 'class 1']
  print("f1_score "+" is :{}%".format(f1_score(y_true=y_true , y_pred= y_pred)))
  print("recall_score "+" is :{}%".format(recall_score(y_true=y_true , y_pred= y_pred)))
  print("precision_score "+" is :{}%".format(precision_score(y_true=y_true , y_pred= y_pred)))
  print(classification_report(y_true, y_pred, target_names=target_names))

In [3]:
data = pd.read_csv('datasets/EDA.csv')
data = data.iloc[: , 1:]

In [6]:
df = data
df = df.drop(columns = ["SalesAmountInEuro","time_delay_for_conversion","click_timestamp","day","day_time"])
df.head(5)

,Sale,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,product_brand,product_category(1),product_category(2),...,product_category(5),product_category(6),product_country,product_id,product_title,partner_id,user_id,day_time_category,tree_encode,category_encode
0,0.0,439.389006,85.491137,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,6,0
1,0.0,439.389006,85.491137,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,6,0
2,0.0,439.389006,85.491137,1,1,0,1,1,1,1,...,0,0,1,2,1,1,2,16,12,3
3,0.0,439.389006,85.491137,0,2,0,0,0,0,0,...,0,0,0,3,0,0,3,20,6,0
4,0.0,439.389006,85.491137,1,0,0,2,2,2,2,...,0,0,2,4,2,2,4,20,18,15


In [1]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# scaling the train and test set
df['nb_clicks_1week'] = sc.fit_transform(df['nb_clicks_1week'])
y = df['Sale'].to_numpy()
X = df.drop(['Sale',"product_price"], axis=1)
X["nb_clicks_1week"]=(X["nb_clicks_1week"]-X["nb_clicks_1week"].mean())/X["nb_clicks_1week"].std()
X.head(5)

NameError: name 'df' is not defined

In [44]:

s_nn = nn.Sequential(
        nn.Linear(20,128),
        nn.LeakyReLU(),
        nn.Linear(128,64),
        nn.LeakyReLU(),
        nn.Linear(64,32),
        nn.LeakyReLU(),
    nn.Dropout(0.4),
        nn.Linear(32,16),
        nn.LeakyReLU(),
        nn.Linear(16,8),
        nn.LeakyReLU(),
        nn.Linear(8,1),
        nn.Sigmoid()

    )
for i in range(20):
    X, y = shuffle(X, y, random_state=i)
from torch.utils.data import DataLoader, TensorDataset
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

s_nn.to(device)

loss_function = nn.BCELoss()

optimizer = torch.optim.SGD(s_nn.parameters(),lr=0.00001)


dataset = TensorDataset(torch.unsqueeze(torch.Tensor(X.to_numpy()),1),torch.Tensor(y))
train_subset, val_subset = torch.utils.data.random_split(
        dataset, [60000, 40000], generator=torch.Generator().manual_seed(1))

BATCH_SIZE = 256
NUMBER_OF_EPOCHS = 50
train_loader = DataLoader(dataset=train_subset, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=val_subset, shuffle=False, batch_size=BATCH_SIZE)

In [45]:
from statistics import mean
all_train_losses = []
all_train_accuracy = []
all_val_losses = []
all_val_accuracy = []
for epoch in range(NUMBER_OF_EPOCHS):
    # training
    epoch_loss = 0
    correct = 0
    acc_list = []
    epoch_all = 0
    s_nn.train()
    with tqdm.tqdm(enumerate(train_loader), total=len(train_loader)) as pbar:
        for i, (x, y) in pbar:
            images , labels = x.to(device) , y.to(device,dtype = torch.float)
            optimizer.zero_grad()
            outputs = s_nn(images)
            outputs = outputs.squeeze(1)
            labels = labels.unsqueeze(1)
            loss = loss_function (outputs , labels)
            loss.backward()
            optimizer.step()
            # calculate accuracy
            epoch_loss += outputs.shape[0] * loss.item()
            epoch_all += labels.size(0)
            predicted = torch.round(outputs)#.argmax(-1)#torch.max(outputs.data,1)[0]
            correct += (predicted == labels).sum().item()
            #print(correct)
            #acc_list.append(correct/total)
            pbar.set_description(f'epoch {epoch } - train Loss: {epoch_loss / (i + 1):.3e} - train Acc: {correct * 100. / epoch_all:.2f}%')
   # all_train_losses.append(epoch_loss/60000)
    ##print(f'trainin loss : {epoch_loss/60000}')
    #print(f'trainin acc : {mean(acc_list)}')
    #all_train_accuracy.append(mean(acc_list))
    loss_list = []
    s_nn.eval() 
    with torch.no_grad():
        main = []
        pre = []
        epoch_loss = 0
        epoch_all = 0
        correct = 0
        corr = 0
        tot = 0
        with tqdm.tqdm(enumerate(val_loader), total=len(val_loader)) as pbar:
            for i, (x, y) in pbar:
                images , labels = x.to(device) , y.to(device,dtype = torch.float)
                out = s_nn(images)
                out = out.squeeze(1)
                labels = labels.unsqueeze(1)
                los = loss_function (out , labels).item()
                main.append(labels)
                
                epoch_loss += los
                loss_list.append(los)
                predicts = torch.round(out)
                pre.append(predicts)
                epoch_all+=labels.size(0)
                tot += labels.size(0)
                correct += (predicts == labels).sum().item()
                pbar.set_description(f'epoch {epoch } - val Loss: {epoch_loss / (i + 1):.3e} - val Acc: {correct * 100. / epoch_all:.2f}%')


epoch 0 - train Loss: 2.117e+03 - train Acc: 85.61%: 100%|█| 235/235 [00:01<00:0
epoch 0 - val Loss: 7.322e+00 - val Acc: 86.25%: 100%|█| 157/157 [00:00<00:00, 2
epoch 1 - train Loss: 1.621e+03 - train Acc: 85.80%: 100%|█| 235/235 [00:01<00:0
epoch 1 - val Loss: 2.501e+00 - val Acc: 86.28%: 100%|█| 157/157 [00:00<00:00, 2
epoch 2 - train Loss: 8.498e+02 - train Acc: 85.86%: 100%|█| 235/235 [00:02<00:0
epoch 2 - val Loss: 8.239e-01 - val Acc: 86.29%: 100%|█| 157/157 [00:01<00:00, 1
epoch 3 - train Loss: 3.752e+02 - train Acc: 86.27%: 100%|█| 235/235 [00:02<00:0
epoch 3 - val Loss: 5.501e-01 - val Acc: 86.29%: 100%|█| 157/157 [00:01<00:00, 1
epoch 4 - train Loss: 2.291e+02 - train Acc: 86.31%: 100%|█| 235/235 [00:02<00:0
epoch 4 - val Loss: 4.871e-01 - val Acc: 86.29%: 100%|█| 157/157 [00:01<00:00, 1
epoch 5 - train Loss: 1.818e+02 - train Acc: 86.33%: 100%|█| 235/235 [00:02<00:0
epoch 5 - val Loss: 4.735e-01 - val Acc: 86.29%: 100%|█| 157/157 [00:01<00:00, 1
epoch 6 - train Loss: 1.572e

In [ ]:
visualize_result(main,pre)

In [ ]:
from utils import convertor as cvn
convertor = cvn.ConvertorD("")